## Project Expat Assistance

Starting with the Web Framework with Flask

In [7]:
#installing the flask

#remove in case flask is not installed
#pip uninstall flask 

#in case upgrade needed
#!pip install --upgrade flask

#flask --version

In [8]:
import sqlite3

## Running Example code with Flask

In [9]:
from flask import Flask, render_template, redirect, url_for  #importing libraries
#from jinja2 import pass_context


app = Flask(__name__)    #__name__ is communicating with Flask function for user's
                         #current directory specification

#routing in FLask
@app.route("/")          #Calling flask to route to the end of a website eg"xcvb.com/" to execute a code
def Landing_Page():         #function Landing Page at/
    try:
        return render_template("home.html")
    except Exception as e:
        return str(e)            #Just showing the error if any

@app.route("/login")
def login():
     try:
        return render_template("login.html")   
     except Exception as e:
        return str(e)

@app.route("/home")
def Home_Page():
     return "Help us to Help You!"

@app.route("/home/apply")
def Application_Page():
     return "Choose your way"

@app.route("/home/apply/trc")
def Apply_TRC():
     return "Submit your response here"    
    
def get_db_connection():
    conn = sqlite3.connect('expat_project.db')
    conn.row_factory = sqlite3.Row
    return conn
    
#page to select a new application or see existing, need to add buttons here to see existing apps or start a new one 
@app.route("/welcome")
def welcome():
    return render_template('welcome.html')

#page and query to get existing cases for a user 

@app.route("/users/<user_id>/cases")
def user_cases(user_id):
    conn = get_db_connection()
    rows = conn.execute("""SELECT  t.title, u.first_name, u.last_name, a.created_at, a.updated_at, c.first_name, c.last_name, c.company_name, a.status 
        FROM applications AS a
        LEFT JOIN application_types as t 
        ON a.application_type_id = t.id
        LEFT JOIN users AS u
        ON a.creator_id = u.id
        LEFT JOIN consultants AS c
        ON a.assigned_consultant_id = c.id
        WHERE a.creator_id = """ + user_id).fetchall()
    conn.close()
    return render_template('user_cases.html', rows = rows) 

#page to start a new application

@app.route("/users/<user_id>/applications/<application_type_id>/create")
def start_application(user_id, application_type_id):
    conn = get_db_connection()
    rows = conn.execute('SELECT title FROM questions WHERE application_type_id = ' + application_type_id ).fetchall()
    conn.close()
    return render_template('start_application.html', rows = rows)  

#page for consulatnts to 

@app.route("/consultants/<consultant_id>/cases")
def consultant_cases(consultant_id):
    conn = get_db_connection()
    rows = conn.execute("""SELECT  a.id, t.title, u.first_name, u.last_name, a.created_at, a.updated_at, c.first_name, c.last_name, c.company_name, a.status
FROM applications as a
LEFT JOIN application_types as t
ON a.application_type_id = t.id
LEFT JOIN users as u
ON 	a.creator_id = u.id
LEFT JOIN consultants as c
ON 	a.assigned_consultant_id = c.id 
WHERE a.assigned_consultant_id =  """ + consultant_id ).fetchall()
    conn.close()
    return render_template('consultant_cases.html', rows = rows) 



app.run(host='localhost', port=8081, debug=True, use_reloader=False) #running app which is Flask at localhost at port 8080



 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


OSError: [Errno 48] Address already in use